In [1]:

from pathlib import Path
import pandas as pd
import os

In [109]:
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data
path = data_path / 'stock price data'
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv") 
ticker_all
stock = pd.read_csv(path/"company stock price daily"/'AAPL.csv') 
stock_bench = pd.read_csv(path/"benchmark price daily"/'^GSPC.csv') 

In [46]:
time_start = "2014-01-01"
time_end = "2016-01-01"
stock_time = stock[(stock["date"]>=time_start)&(stock["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)
stock_bench_time = stock_bench[(stock_bench["date"]>=time_start)&(stock_bench["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)

In [49]:
stock_time

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Symbol
0,2014-01-02,19.8457,19.8939,19.7150,19.7546,17.364962,235167828,8398851,-0.0911,-0.45904,19.77,"January 02, 14",-0.004590,AAPL
1,2014-01-03,19.7450,19.7750,19.3011,19.3207,16.983521,393215480,14043410,-0.4243,-2.15000,19.47,"January 03, 14",-0.021500,AAPL
2,2014-01-06,19.1946,19.5286,19.0571,19.4261,17.076143,413436604,14765593,0.2315,1.21000,19.32,"January 06, 14",0.012100,AAPL
3,2014-01-07,19.4400,19.4986,19.2116,19.2871,16.954014,317731064,11347538,-0.1529,-0.78652,19.33,"January 07, 14",-0.007865,AAPL
4,2014-01-08,19.2432,19.4843,19.2389,19.4093,17.061382,258746656,9240952,0.1661,0.86316,19.40,"January 08, 14",0.008632,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2015-12-24,27.2500,27.2500,26.9875,27.0075,24.653078,54386720,13596680,-0.2425,-0.88991,27.09,"December 24, 15",-0.008899,AAPL
500,2015-12-28,26.8975,26.9225,26.5452,26.7050,24.376947,106744840,26686210,-0.1925,-0.71568,26.71,"December 28, 15",-0.007157,AAPL
501,2015-12-29,26.7400,27.3575,26.7150,27.1850,24.815102,123724788,30931197,0.4450,1.66000,27.09,"December 29, 15",0.016600,AAPL
502,2015-12-30,27.1450,27.1750,26.7950,26.8300,24.491055,100855108,25213777,-0.3150,-1.16000,26.91,"December 30, 15",-0.011600,AAPL


In [112]:
def volatility(stock, stock_bench, method="std", period=None):
    # Ensure the 'date' column is in datetime format
    df = stock.copy()
    df['date'] = pd.to_datetime(df['date'])
    # Set 'date' as the index
    df.set_index('date', inplace=True)

    df_bench = stock_bench.copy()
    df_bench['date'] = pd.to_datetime(df_bench['date'])

    # Use the entire dataset length as the period if not specified
    if period is None:
        period = len(df)
    
    if method == "std":
        # Calculate the rolling standard deviation, using a minimum of 1 to avoid division by zero
        rolling_stats = df['changePercent'].rolling(window=period).std()
    if method == "range":
        rolling_stats = (df['close'].rolling(window=period, min_periods=1).max() - df['close'].rolling(window=period).min())/df['close'].rolling(window=period).mean()
    if method == "beta":
        # Calculate covariance between stock and market returns
        covariance = pd.concat([df["changePercent"].reset_index(drop=True).rename("Stock return"), df_bench["changePercent"].rename("Market return")], axis=1).rolling(window=30).cov(pairwise=True).unstack()["Stock return", "Market return"]
        # Calculate variance of market returns
        market_variance = df_bench['changePercent'].rolling(window=period).var()
        rolling_stats = covariance / market_variance
    return rolling_stats.mean()

In [113]:
ticker_all

,Symbol,Sector,Industry
0,A,Healthcare,Diagnostics & Research
1,AAL,Industrials,Airlines
2,AAN,Industrials,Rental & Leasing Services
3,AAON,Industrials,Building Products & Equipment
4,AAP,Consumer Cyclical,Specialty Retail
...,...,...,...
2112,ZEUS,Basic Materials,Steel
2113,ZION,Financial Services,Banks—Regional
2114,ZIONP,Financial Services,Banks—Regional
2115,ZUMZ,Consumer Cyclical,Apparel Retail


In [121]:
time_start = "2014-01-01"
time_end = "2016-01-01"
# ['Healthcare', 'Industrials', 'Consumer Cyclical', 'Technology','Financial Services', 'Real Estate', 'Basic Materials','Consumer Defensive', 'Energy', 'Utilities','Communication Services']
Sectors = ['Industrials', 'Consumer Cyclical', 'Technology','Financial Services', 'Real Estate', 'Basic Materials', 'Consumer Defensive', 'Communication Services']
ticker_sector = ticker_all[ticker_all["Sector"].isin(Sectors)]
ticker_dict = {}
top = 10
for ticker in ticker_sector["Symbol"].unique():
    ticker_file = ticker+".csv"
    stock = pd.read_csv(path/"company stock price daily"/ticker_file)
    stock_time = stock[(stock["date"]>=time_start)&(stock["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)
    stock_bench_time = stock_bench[(stock_bench["date"]>=time_start)&(stock_bench["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)
    ticker_dict[ticker] = volatility(stock_time, stock_bench_time, method="std", period=30)
top_ticker = [item[0] for item in sorted(ticker_dict.items(), key=lambda x: x[1])[:top]]

In [122]:
top_ticker

['DDT', 'BKT', 'ELA', 'VKQ', 'BTA', 'CHT', 'EVV', 'LEO', 'HSBC', 'DSM']